In [4]:

from roboflow import Roboflow
rf = Roboflow(api_key="htpcxp3XQh7SsgMfjJns")
project = rf.workspace("ownprojects").project("basketball-w2xcw")
dataset = project.version(2).download("yolov8")


loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.0.178, to fix: `pip install ultralytics==8.0.196`


KeyboardInterrupt: 

In [5]:
import torch
from torch import nn

# Note: this notebook requires torch >= 1.10.0
torch.__version__

'2.0.1+cpu'

In [6]:
# Setup device-agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [7]:
import os
from pathlib import Path

# the path to BasketBall-2 dataset is object_detection_roboflow\validation_notebooks\Basketball-2
#what path does Path() return?
# Path() returns the current working directory
# Path.cwd() returns the current working directory
# Path.home() returns the home directory

print(Path.cwd())
print(Path.home())
print(Path('Basketball-2'))

image_path = Path('Basketball-2')


def walk_through_dir(dir_path):
  """
  Walks through dir_path returning its contents.
  Args:
    dir_path (str or pathlib.Path): target directory

  Returns:
    A print out of:
      number of subdiretories in dir_path
      number of images (files) in each subdirectory
      name of each subdirectory
  """
  for dirpath, dirnames, filenames in os.walk(dir_path):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")


walk_through_dir(image_path)

c:\Users\ghadf\OneDrive\Desktop\Data Analytics\Python\DL\git\Deep_learning\object_detection_roboflow\validation_notebooks
C:\Users\ghadf
Basketball-2


In [8]:
import random
from PIL import Image

# Set seed
random.seed(42) # <- try changing this and see what happens

# 1. Get all image paths (* means "any combination")
image_path_list = list(image_path.glob("*/*/*.jpg"))

# 2. Get random image path
random_image_path = random.choice(image_path_list)

# 3. Get image class from path name (the image class is the name of the directory where the image is stored)
image_class = random_image_path.parent.stem

# 4. Open image
img = Image.open(random_image_path)

# 5. Print metadata
print(f"Random image path: {random_image_path}")
print(f"Image class: {image_class}")
print(f"Image height: {img.height}")
print(f"Image width: {img.width}")
img

IndexError: Cannot choose from an empty sequence

add in a bounding box checker for before and after transformations

In [9]:
import os
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from PIL import Image


# Define a consistent transform for images
def get_transform(train):
    transforms_list = [transforms.ToTensor()]
    if train:
        transforms_list.extend([
            transforms.Resize((224, 224), antialias=True),
            transforms.RandomHorizontalFlip(0.5)
        ])
    else:
        transforms_list.append(transforms.Resize((224, 224), antialias=True))
    return transforms.Compose(transforms_list)


In [10]:
def verify_and_correct_boxes(boxes, image_width, image_height):
    corrected_boxes = []
    for box in boxes:
        # Extract coordinates
        x_center, y_center, box_width, box_height = box

        # Convert centers to top-left coordinates
        x_min = (x_center - box_width / 2) * image_width
        y_min = (y_center - box_height / 2) * image_height

        # Ensure width and height are positive
        box_width = abs(box_width * image_width)
        box_height = abs(box_height * image_height)

        # Convert to bottom-right coordinates
        x_max = x_min + box_width
        y_max = y_min + box_height

        # Clip boxes that are out of image bounds
        x_min = max(0, min(x_min, image_width - 1))
        y_min = max(0, min(y_min, image_height - 1))
        x_max = max(x_min + 1, min(x_max, image_width))
        y_max = max(y_min + 1, min(y_max, image_height))

        corrected_boxes.append((x_min, y_min, x_max, y_max))

    return torch.tensor(corrected_boxes, dtype=torch.float32)


In [11]:
class CustomDataset(Dataset):
    def __init__(self, img_dir, label_dir, transform=None):
        self.img_dir = img_dir
        self.label_dir = label_dir
        self.transform = transform
        self.img_names = [img_name for img_name in os.listdir(img_dir) if img_name.endswith('.jpg')]

    def load_labels(self, label_path):
        with open(label_path, 'r') as file:
            labels = file.read().splitlines()

        annotations = []
        for label in labels:
            elements = label.split()
            # Only process labels with exactly 5 elements
            if len(elements) == 5:
                class_label, x_center, y_center, width, height = map(float, elements)
                annotations.append((class_label, x_center, y_center, width, height))
            else:
                print(f"Skipping invalid label in {label_path}: {label}")
        return annotations

    def __len__(self):
        return len(self.img_names)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_names[idx])
        label_path = os.path.join(self.label_dir, self.img_names[idx].replace('.jpg', '.txt'))
        image = Image.open(img_path).convert('RGB')

        # Store original image size before transformation
        orig_width, orig_height = image.size

        # Apply transformations
        if self.transform:
            image = self.transform(image)

        # If your transforms include resizing, then you should get the new size from the transformed image
        if isinstance(image, torch.Tensor):
            # For tensors, the size is in the shape attribute
            img_height, img_width = image.shape[-2:]
        else:
            # If the image is not a tensor, it's still a PIL image, and we can use the size method
            img_width, img_height = image.size

        annotations = self.load_labels(label_path)
        boxes = [ann[1:] for ann in annotations]  # Extract only the bounding box coordinates
        corrected_boxes = verify_and_correct_boxes(boxes, orig_width, orig_height)  # Use the original size here for correction

        labels = [ann[0] for ann in annotations]  # Extract only the labels
        labels = torch.tensor(labels, dtype=torch.int64)  # Convert labels to tensor

        image_id = torch.tensor([idx])
        area = (corrected_boxes[:, 3] - corrected_boxes[:, 1]) * (corrected_boxes[:, 2] - corrected_boxes[:, 0])
        iscrowd = torch.zeros((len(annotations),), dtype=torch.int64)

        target = {
            "boxes": corrected_boxes,
            "labels": labels,
            "image_id": image_id,
            "area": area,
            "iscrowd": iscrowd
        }

        return image, target


In [12]:
# Set up the dataset and dataloader
train_data = CustomDataset(
    img_dir='Basketball-2/train/images',
    label_dir='Basketball-2/train/labels',
    transform=get_transform(train=True)
)

test_data = CustomDataset(
    img_dir='Basketball-2/test/images',
    label_dir='Basketball-2/test/labels',
    transform=get_transform(train=False)
)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'Basketball-2/train/images'

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

train_dataloader = DataLoader(
    train_data, batch_size=2, shuffle=True, collate_fn=collate_fn, num_workers=2
)
test_dataloader = DataLoader(
    test_data, batch_size=2, shuffle=False, collate_fn=collate_fn, num_workers=2
)

In [20]:
# Model setup
num_classes = 6  # 5 classes + background

model = fasterrcnn_resnet50_fpn(pretrained=True)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
model.to(device)

The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [13]:
from torchvision.ops import box_iou

 #New IoU based metrics calculation function
def calculate_iou_metrics(pred_boxes, true_boxes, iou_threshold=0.5):
    true_positives = 0
    false_positives = 0
    false_negatives = 0

    for pred, true in zip(pred_boxes, true_boxes):
        # Check if either predicted or true boxes are empty
        if pred['boxes'].nelement() == 0 or true['boxes'].nelement() == 0:
            if pred['boxes'].nelement() == 0 and true['boxes'].nelement() != 0:
                false_negatives += len(true['boxes'])
            continue

        iou = box_iou(pred['boxes'], true['boxes'])

        for i in range(len(pred['boxes'])):
            if iou[i].max() > iou_threshold:
                true_positives += 1
            else:
                false_positives += 1

        false_negatives += len(true['boxes']) - iou.max(dim=0)[0].sum().item()

    precision = true_positives / (true_positives + false_positives) if true_positives + false_positives > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if true_positives + false_negatives > 0 else 0
    return precision, recall


In [14]:
def calculate_accuracy(pred_boxes, true_boxes, iou_threshold=0.5):
    accuracy = 0.0
    for pred, true in zip(pred_boxes, true_boxes):
        iou = box_iou(pred['boxes'], true['boxes'])
        correct_predictions = iou > iou_threshold

        # Check for division by zero
        if correct_predictions.numel() == 0:
            continue

        accuracy += correct_predictions.sum().item() / correct_predictions.numel()
    return accuracy / len(pred_boxes) if len(pred_boxes) > 0 else 0.0

In [15]:
# Training function remains the same
def train_one_epoch(model, optimizer, data_loader, device, epoch):
    model.train()
    train_loss = 0.0
    train_accuracy = 0.0
    for batch, (images, targets) in enumerate(data_loader):
        images = list(img.to(device) for img in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        if not torch.isfinite(losses):
            print("Loss is not finite, stopping training.")
            return None

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        train_loss += losses.item()
        train_accuracy += calculate_accuracy(targets, targets)

    train_loss /= len(data_loader)
    train_accuracy /= len(data_loader)

    print(f"train_loss: {train_loss}, train_accuracy: {train_accuracy}")
    return train_loss, train_accuracy



In [16]:
# Modified evaluate function
def evaluate(model, data_loader, device):
    model.eval()
    test_precision = 0.0
    test_recall = 0.0
    with torch.no_grad():
        for images, targets in data_loader:
            images = list(img.to(device) for img in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
            preds = model(images)

            precision, recall = calculate_iou_metrics(preds, targets)
            test_precision += precision
            test_recall += recall

    test_precision /= len(data_loader)
    test_recall /= len(data_loader)

    print(f"test_precision: {test_precision}, test_recall: {test_recall}")
    return test_precision, test_recall


In [17]:
# Adjusted train and evaluate loop
def train_and_evaluate(model, train_dataloader, test_dataloader, optimizer, lr_scheduler, device, num_epochs):
    train_losses = []
    test_iou_scores = []

    for epoch in range(num_epochs):
        train_loss, train_accuracy = train_one_epoch(model, optimizer, train_dataloader, device, epoch)
        if train_loss is None:
            print("Training stopped due to an error.")
            break

        test_precision, test_recall = evaluate(model, test_dataloader, device)
        if test_precision is None or test_recall is None:
            print("Evaluation stopped due to an error.")
            break

        lr_scheduler.step()

        train_losses.append(train_loss)
        test_iou_scores.append((test_precision, test_recall))

        print(f"Epoch {epoch+1}/{num_epochs} - Train loss: {train_loss}, Train accuracy: {train_accuracy}, Test precision: {test_precision}, Test recall: {test_recall}")

    return train_losses, test_iou_scores



In [18]:
# Simplified training loop setup
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
num_epochs = 10

# Train the model and get metrics
train_losses, test_iou_scores = train_and_evaluate(model, train_dataloader, test_dataloader, optimizer, lr_scheduler, device, num_epochs)


# Call the train_and_evaluate function
#train_and_evaluate(model, train_dataloader, test_dataloader, optimizer, lr_scheduler, device, num_epochs)

NameError: name 'model' is not defined

Looking at training and test loss curves is a great way to see if your model is overfitting.

An overfitting model is one that performs better (often by a considerable margin) on the training set than the validation/test set.

If your training loss is far lower than your test loss, your model is overfitting.

As in, it's learning the patterns in the training too well and those patterns aren't generalizing to the test data.

The other side is when your training and test loss are not as low as you'd like, this is considered underfitting.

The ideal position for a training and test loss curve is for them to line up closely with each other

In [ ]:
import matplotlib.pyplot as plt

# Plotting the training loss
plt.figure(figsize=(12, 6))
plt.plot(train_losses, label='Training Loss')
plt.title('Training Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Plotting the testing IoU
plt.figure(figsize=(12, 6))
plt.plot(test_iou_scores, label='Testing IoU Score', color='orange')
plt.title('Testing IoU Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('IoU Score')
plt.legend()
plt.show()


In [25]:
import torch

#load to infer on new images
model.load_state_dict(torch.load('model_state_dict.pth'), map_location=torch.device('cpu'))
model.eval()
model.to(device)

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [ ]:
import requests
from PIL import Image
from io import BytesIO
import os

# Replace this with the direct URL of the image you want to download
image_url = "https://www.google.com/search?client=firefox-b-1-d&sca_esv=3327f1c9d2227784&sxsrf=AM9HkKlt52SlfwUmlCipyy1q9dCgaGxKZA:1701532475460&q=stephen+curry+shooting+threes&tbm=isch&source=lnms&sa=X&ved=2ahUKEwiA5Ze2jvGCAxWcVTABHZunDX8Q0pQJegQIDBAB&biw=1600&bih=739&dpr=1#imgrc=qPWrl1aHYQs17M"
image_url = "https://www.google.com/url?sa=i&url=https%3A%2F%2Fwww.sbnation.com%2Fnba%2F2017%2F3%2F6%2F14823068%2F2017-nba-scores-steph-curry-gobert-ulis-westbrook&psig=AOvVaw3BAtfjqcv949FdTA2tM5yb&ust=1701618880377000&source=images&cd=vfe&opi=89978449&ved=0CBIQjRxqFwoTCMCQwbqO8YIDFQAAAAAdAAAAABAE"
image_url = "https://cdn.vox-cdn.com/thumbor/nPF0yG3Mi4c09o4MfRag0bLi8_g=/0x0:4497x2998/920x613/filters:focal(1280x131:1998x849):format(webp)/cdn.vox-cdn.com/uploads/chorus_image/image/53550403/usa_today_9920467.0.jpg"


# Fetch the image from the URL
response = requests.get(image_url)
if response.status_code == 200:
    image = Image.open(BytesIO(response.content))

    # Path to save the image in Colab's local storage
    folder_path = '/content/inference_images/'  # Local directory in Google Colab
    os.makedirs(folder_path, exist_ok=True)

    # Save the image
    file_path = os.path.join(folder_path, 'stephen_curry_shooting.jpeg')
    image.save(file_path)

    print(f"Image saved at {file_path}")
else:
    print(f"Failed to download the image. Status code: {response.status_code}")



In [ ]:
from torchvision import transforms
from PIL import Image

# Define the transform
def get_transform():
    return transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((224, 224)),  # Resize if necessary
    ])

# Load an image
image_path = '/content/inference_images/stephen_curry_shooting.jpeg'
img = Image.open(image_path).convert('RGB')
transformed_img = get_transform()(img).to(device)


In [ ]:
with torch.no_grad():
    prediction = model([transformed_img])


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Function to display image with bounding boxes
def display_image_with_boxes(image, boxes, labels, scores, threshold=0.5):
    # Convert image tensor to numpy
    image = image.cpu().numpy().transpose(1, 2, 0)
    fig, ax = plt.subplots(1, figsize=(12, 9))
    ax.imshow(image)

    for box, label, score in zip(boxes, labels, scores):
        if score > threshold:
            x, y, x2, y2 = box
            bbox = patches.Rectangle((x, y), x2 - x, y2 - y, linewidth=2, edgecolor='r', facecolor='none')
            ax.add_patch(bbox)
            plt.text(x, y, f'Class: {label}, Score: {score:.2f}', bbox=dict(facecolor='yellow', alpha=0.5))

    plt.show()

# Process the prediction
pred_boxes = [box.cpu().numpy() for box in prediction[0]['boxes']]
pred_labels = [label.cpu().numpy() for label in prediction[0]['labels']]
pred_scores = [score.cpu().numpy() for score in prediction[0]['scores']]

# Display the image
display_image_with_boxes(transformed_img, pred_boxes, pred_labels, pred_scores)
